In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28
# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [4]:
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

In [5]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [6]:
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [7]:
def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)

def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

In [8]:
def accuracy(y_pred, y_true):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.optimizers.SGD(learning_rate)

In [9]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    gradients = g.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [10]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 1227.489014, accuracy: 0.691406
step: 100, loss: 1233.890869, accuracy: 0.664062
step: 150, loss: 1129.556885, accuracy: 0.699219
step: 200, loss: 656.273499, accuracy: 0.839844
step: 250, loss: 648.874573, accuracy: 0.851562
step: 300, loss: 544.163086, accuracy: 0.878906
step: 350, loss: 546.070801, accuracy: 0.835938
step: 400, loss: 500.096497, accuracy: 0.855469
step: 450, loss: 666.262573, accuracy: 0.828125
step: 500, loss: 759.292114, accuracy: 0.812500
step: 550, loss: 637.896240, accuracy: 0.835938
step: 600, loss: 447.092468, accuracy: 0.894531
step: 650, loss: 717.965454, accuracy: 0.820312
step: 700, loss: 657.527710, accuracy: 0.828125
step: 750, loss: 456.258301, accuracy: 0.871094
step: 800, loss: 655.809570, accuracy: 0.839844
step: 850, loss: 62.434891, accuracy: 0.917969
step: 900, loss: 91.523392, accuracy: 0.886719
step: 950, loss: 34.892048, accuracy: 0.968750
step: 1000, loss: 105.454239, accuracy: 0.902344


In [11]:
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.896100
